# UFO

<font color='#660066'>
   
### Table of Contents

- [Data](#Data)
    - [Scraping from UFORC](#Scraping-from-UFORC)
    - [Data Format](#data_format)
    - [Cleaing Data](#cleaning_pdata)
- [Basic anlayis](#Basic-Anylsis)
- [Milestone 3](#milestones)

In [1]:
%load_ext autoreload
%autoreload 2
import json
import pandas as pd
from lib.scraping import fetch_shape_urls,  \
                     fetch_all_sighting_urls, \
                     store_to_json, \
                     build_report_dataframe
from lib.postprocessing import get_time_of_occurrence, \
                           to_datetime_add_year, \
                           replace_empty_with_nan, \
                           distribution_histogram 
from lib.cleaning import clean_data, \
                        clean_duration_get_min
from lib.nuforc_analysis import *

In [2]:
data_folder = "data/"

file_url_list = data_folder+'full_list_of_urls.json'

report_file = 'jurgis_full.csv'

<font color='#660066'>
    
## Loading data

of course we dont want so scrape every time

In [3]:
df_reports = pd.read_csv(data_folder+report_file)
df_reports.sample(1).head()

,Unnamed: 0,Duration,Location,Occurred,Posted,Reported,Shape,Summary,url
22105,14216,45 minutes,"Myrtle Beach, SC",6/10/2014 22:30 (Entered as : 06/10/14 22:30),6/13/2014,6/10/2014 8:12:50 PM 20:12,Circle,"['Orange lights over water.', ""Sitting on 6th ...",http://www.nuforc.org/webreports/110/S110121.html


In [4]:
df_reports, df_madarr = clean_data(df_reports)

In [5]:
df_reports.head()

,Duration,Location,State,Occurred,Posted,Reported,Shape,Summary,nuforc_note,link
url,,,,,,,,,,
http://www.nuforc.org/webreports/143/S143627.html,10.000,Lutz,FL,2018-10-30 23:30:00,11/9/2018,2018-10-30 20:54:00,,['Blinking white light that moved every few se...,[((NUFORC Note: We have attempted to contact ...,[]
http://www.nuforc.org/webreports/089/S89377.html,4.500,Bridgewater,NJ,2011-11-23 00:45:00,6/5/2012,2012-06-05 12:37:00,,['EXPLAINED: Bright lights in Somerset county ...,[((NUFORC Note: Witness elects to remain tota...,[http://www.nj.com/news/index.ssf/2011/11/brig...
http://www.nuforc.org/webreports/019/S19862.html,2.000,South Fork,CO,2001-10-04 14:00:00,3/21/2003,2001-10-15 13:27:00,Cigar,"['Shinny Cigar shape in sky on clear day.', 'D...",[],[]
http://www.nuforc.org/webreports/121/S121007.html,10.000,Hattiesburg,MS,2014-12-17 20:00:00,8/13/2015,2015-08-09 21:11:00,Triangle,['I saw a triangular UFO with red and white li...,[],[]
http://www.nuforc.org/webreports/116/S116071.html,0.125,LaBelle,FL,2014-12-17 17:19:00,12/18/2014,2014-12-17 18:21:00,Triangle,"['White Triangle in South Florida', 'My co-wor...",[],[]


## Scrapping Air-base, rocket launch locations

### Nevermind, Kiru will do it with VIM, too good for me

# Getting some weather shit in this place

In [140]:
!ls data/weather/

StormEvents_details-ftp_v1.0_d1950_c20170120.csv.gz
StormEvents_locations-ftp_v1.0_d1996_c20170717.csv.gz


In [129]:
storm_folder = 'data/weather/'

In [143]:
storms = pd.read_csv(storm_folder+'StormEvents_details-ftp_v1.0_d1950_c20170120.csv.gz')
#storms = pd.read_csv(storm_folder+'StormEvents_locations-ftp_v1.0_d1996_c20170717.csv.gz')

In [145]:
storms.shape

(223, 51)

In [25]:
df_rocket_launches = pd.read_csv(data_folder + 'rocket_launch.csv', sep=';')

In [28]:
df_air_bases = pd.read_csv(data_folder + 'air_bases.csv')

In [29]:
df_air_bases

,base_name,latitude,longitude
0,Altus Air Force Base,34°39′59″N,099°16′05″W
1,Arnold Air Force Base,35°23′33″N,086°05′09″W
2,Barksdale Air Force Base,32°30′07″N,093°39′46″W
3,Beale Air Force Base,39°08′10″N,121°26′11″W
4,Buckley Air Force Base,39°42′06″N,104°45′06″W
5,Cannon Air Force Base,34°22′58″N,103°19′20″W
6,Cavalier Air Force Station,48°43′55″N,97°54′16″W
7,Columbus Air Force Base,33°38′38″N,088°26′38″W
8,Creech Air Force Base,36°35′14″N,115°40′24″W
9,Davis–Monthan Air Force Base,32°09′59″N,110°52′59″W
